# Imports

In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [2]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data,columns=boston_dataset.feature_names)
features = data.drop(['INDUS','AGE'],axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(data=log_prices,columns=['PRICE'])
target.shape

(506, 1)

In [3]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

# property_stats = np.ndarray(shape=(1,11)) # Empty array
# property_stats[0][CRIME_IDX] = features['CRIM'].mean()
# property_stats[0][ZN_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()
property_stats = features.mean().values.reshape(1,11)
property_stats.shape


(1, 11)

In [4]:
features.head()

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33


In [5]:
regr = LinearRegression().fit(features,target)
fitted_vals = regr.predict(features) # fitted values or $y hat$

MSE = mean_squared_error(target,fitted_vals)
RMSE = np.sqrt(MSE)
r_squared = regr.score(features,target)

In [6]:
def get_log_estimate(nr_rooms, students_per_classroom, next_to_river=False, high_confidence=True):
    # Configure Property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    # Make Predictions
    log_estimate = regr.predict(property_stats)[0][0]

    # Calculate Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
        
    return log_estimate, upper_bound, lower_bound, interval


In [7]:
get_log_estimate(5,25, next_to_river=True, high_confidence=False)

(2.7709742654894485, 2.9584864006865788, 2.583462130292318, 68)

In [8]:
np.median(boston_dataset.target)

21.2

In [9]:
CURRENT_MEDIAN_PRICE = 583.3
SCALE_FACTOR = CURRENT_MEDIAN_PRICE / np.median(boston_dataset.target)

In [10]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    
    """
    Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classroom for the school in the area
    chas -- True if the property is next to the river, False otherwise
    large_range -- True for a 95% prediction interval, False for a 68% interval
    
    """
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return
    
    
    log_est, upper, lower, conf = get_log_estimate(nr_rooms=rm, students_per_classroom=ptratio,
                                               next_to_river=chas, high_confidence=large_range)

    # Convert to today's dollars
    dollar_est = np.e**log_est*1000*SCALE_FACTOR
    high_est = np.e**upper*1000*SCALE_FACTOR
    low_est = np.e**lower*1000*SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(high_est, -3)
    rounded_low = np.around(low_est, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [11]:
get_dollar_estimate(rm=3,ptratio=30,chas=True)

The estimated property value is 304000.0.
At 95% confidence the valuation range is
USD 209000.0 at the lower end to USD 442000.0 at the high end.
